In [1]:
from setup import setup_src_path
print(setup_src_path())
import data.processed as processed
import config.config as config
import utils.setup as setup
import utils.functions as fn
from importlib import reload



['/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/notebooks', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages', '/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/modules']


/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-04-20 05:04:18.375287: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-20 05:04:18.408100: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFl

In [15]:
from datasets import load_from_disk

source_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/source_data")
source_data_eval=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/source_data_eval")
target_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/target_data")
target_data_eval=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/target_data_eval")
test_target_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/test_target_data")
unsupervised_target=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/unsupervised_target")


In [3]:
source_data = source_data.train_test_split(train_size=0.5, test_size=0.08)
source_data

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 37506
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 6002
    })
})

In [16]:
from torch.utils.data import DataLoader
source_data_loader = DataLoader(source_data, batch_size=32, shuffle=True)
target_data_loader = DataLoader(target_data, batch_size=32)
source_data_eval_loader = DataLoader(source_data_eval, batch_size=32, shuffle=True)
target_data_eval_loader = DataLoader(target_data_eval, batch_size=32)

target_test_loader = DataLoader(test_target_data, batch_size=32)

In [17]:
from adapters import AutoAdapterModel,init
from tqdm import tqdm

from transformers import AutoConfig, AutoModelForMaskedLM

mdlcfg = AutoConfig.from_pretrained(
    config.Config.MODEL_NAME,
 
)
model = AutoAdapterModel.from_pretrained(
    config.Config.MODEL_NAME,
)
reload(fn)
fn.print_trainable_parameters(model)

trainable params: 66985530 || all params: 66985530 || trainable%: 100.0


In [18]:
import adapters.composition as ac

#domain_adapter_coral = model.load_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/domain_adapter_telephone_travel",with_head=False)
source_mlm = model.load_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/source-mlm",with_head=False)
target_mlm = model.load_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/standalone-mlm-target",with_head=False)


In [19]:
model.add_adapter_fusion(ac.Fuse(source_mlm, target_mlm))
adapter_setup_fuse = ac.Fuse( source_mlm, target_mlm)
model.train_adapter_fusion(adapter_setup_fuse)

#model.active_adapters = ac.Stack(domain_adapter_coral, ac.Fuse(source_mlm, target_mlm))
model.active_adapters = ac.Fuse(source_mlm, target_mlm)

model.add_classification_head("mnli", num_labels=3)


In [20]:
fn.print_trainable_parameters(model)

trainable params: 11811075 || all params: 85296189 || trainable%: 13.847130966191234


In [21]:
model.adapter_summary(as_dict=True)

[{'name': 'source-mlm',
  'architecture': 'bottleneck',
  'active': True,
  '#param': 3545856,
  'train': False,
  '%param': 4.605670051672885},
 {'name': 'standalone-mlm-target',
  'architecture': 'bottleneck',
  'active': True,
  '#param': 3545856,
  'train': False,
  '%param': 4.605670051672885},
 {'name': 'Full model', '#param': 76988928, '%param': 100.0, 'train': False}]

In [22]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)
print(model.active_adapters)

distilbert.transformer.layer.0.output_adapters.adapter_fusion_layer.source-mlm,standalone-mlm-target.query.weight
distilbert.transformer.layer.0.output_adapters.adapter_fusion_layer.source-mlm,standalone-mlm-target.query.bias
distilbert.transformer.layer.0.output_adapters.adapter_fusion_layer.source-mlm,standalone-mlm-target.key.weight
distilbert.transformer.layer.0.output_adapters.adapter_fusion_layer.source-mlm,standalone-mlm-target.key.bias
distilbert.transformer.layer.0.output_adapters.adapter_fusion_layer.source-mlm,standalone-mlm-target.value.weight
distilbert.transformer.layer.1.output_adapters.adapter_fusion_layer.source-mlm,standalone-mlm-target.query.weight
distilbert.transformer.layer.1.output_adapters.adapter_fusion_layer.source-mlm,standalone-mlm-target.query.bias
distilbert.transformer.layer.1.output_adapters.adapter_fusion_layer.source-mlm,standalone-mlm-target.key.weight
distilbert.transformer.layer.1.output_adapters.adapter_fusion_layer.source-mlm,standalone-mlm-target

In [23]:
# Evaluate the model on the target domain before adaptation
reload(fn)
accuracy_before, f1_before = fn.evaluate_model(model, target_test_loader)
print(f"Accuracy before adaptation: {accuracy_before}")
print(f"F1 score before adaptation: {f1_before}")

Accuracy before adaptation: 0.3476720647773279
F1 score before adaptation: 0.20148890096508978


In [24]:

reload(fn)
trainer = fn.train_model(model,"fused_domains",source_data,source_data_eval)

/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


  0%|          | 0/9380 [00:00<?, ?it/s]

{'loss': 1.0122, 'learning_rate': 0.0001, 'epoch': 0.21}


  0%|          | 0/242 [00:00<?, ?it/s]

{'eval_loss': 0.8864644765853882, 'eval_accuracy': 0.5906916612798966, 'eval_f1': 0.5733483197334996, 'eval_precision': 0.6684059715754422, 'eval_recall': 0.5906916612798966, 'eval_runtime': 16.2893, 'eval_samples_per_second': 474.85, 'eval_steps_per_second': 14.856, 'epoch': 0.21}
{'loss': 0.8548, 'learning_rate': 9.436936936936938e-05, 'epoch': 0.43}


...

{'eval_loss': 0.7867714762687683, 'eval_accuracy': 0.659082094376212, 'eval_f1': 0.658798455846046, 'eval_precision': 0.6813040809836378, 'eval_recall': 0.659082094376212, 'eval_runtime': 16.9316, 'eval_samples_per_second': 456.837, 'eval_steps_per_second': 14.293, 'epoch': 3.62}
{'loss': 0.721, 'learning_rate': 4.279279279279279e-06, 'epoch': 3.84}


  0%|          | 0/242 [00:00<?, ?it/s]

{'eval_loss': 0.7791407704353333, 'eval_accuracy': 0.6625727213962508, 'eval_f1': 0.6618674068135038, 'eval_precision': 0.6835035059239345, 'eval_recall': 0.6625727213962508, 'eval_runtime': 16.9813, 'eval_samples_per_second': 455.501, 'eval_steps_per_second': 14.251, 'epoch': 3.84}


Overwriting existing adapter 'source-mlm'.
Overwriting existing adapter 'standalone-mlm-target'.
Overwriting existing adapter fusion module 'source-mlm,standalone-mlm-target'


{'train_runtime': 1680.7334, 'train_samples_per_second': 178.524, 'train_steps_per_second': 5.581, 'train_loss': 0.7812599946695096, 'epoch': 4.0}


In [25]:
# Evaluate the model on the target domain before adaptation
reload(fn)
accuracy_before, f1_before = fn.evaluate_model(trainer.model, target_test_loader)
print(f"Accuracy before adaptation: {accuracy_before}")
print(f"F1 score before adaptation: {f1_before}")

Accuracy before adaptation: 0.6523279352226721
F1 score before adaptation: 0.6537088802705093


In [17]:
# Evaluate the model on the target domain before adaptation
#this is using the full sourcec domain data
reload(fn)
accuracy_before, f1_before = fn.evaluate_model(trainer.model, target_test_loader)
print(f"Accuracy before adaptation: {accuracy_before}")
print(f"F1 score before adaptation: {f1_before}")

Accuracy before adaptation: 0.7211538461538461
F1 score before adaptation: 0.7214558115652716


In [40]:
# Evaluate the model on the target domain before adaptation
reload(fn)
accuracy_before, f1_before = fn.evaluate_model(trainer.model, target_test_loader)
print(f"Accuracy before adaptation: {accuracy_before}")
print(f"F1 score before adaptation: {f1_before}")

Accuracy before adaptation: 0.7130566801619433
F1 score before adaptation: 0.7149068389081805
